## Notebook BQ with Pandas

### Importando Bibliotecas

In [ ]:
from google.cloud import bigquery
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
client = bigquery.Client()
dataset = 'dataset_2011*'

## Verificando o Período

In [ ]:
sql="""
SELECT
  MIN(DATA) AS PRIMEIRA_COMPRA,
  MAX(DATA) AS ULTIMA_COMPRA
FROM (
  SELECT
    PARSE_DATE('%d/%m/%Y',SPLIT(DT_CONFIRMACAO, ' ')[OFFSET(0)]) AS DATA
  FROM
    `{}`)""".format(dataset)
query_job = client.query(sql).to_dataframe()
query_job

## Verificando Tipos de Fornecedores

In [ ]:
sql="""
SELECT
  TIPO_FORNECEDOR,
  count(TIPO_FORNECEDOR) AS NUMERO_TOTAL
FROM
 `{}`
GROUP BY
  TIPO_FORNECEDOR""".format(dataset)
query_job = client.query(sql).to_dataframe()
query_job

## Verificando Número Total de Reservas

In [ ]:
sql="""
SELECT
  COUNT(DISTINCT CD_RESERVA) AS NUMERO_TOTAL_RESERVAS
FROM
  `{}`""".format(dataset)
query_job = client.query(sql).to_dataframe()
query_job

## Calculando a Valor Total dos Recibos

In [ ]:
Valor_total_recibos="""
SELECT
  SUM(VL_TOTAL_RECIBO_) AS VALOR_TOTAL_RECIBOS
FROM(
SELECT
  NR_RECIBO_SYSTUR,
  MAX(VL_TOTAL_RECIBO) AS VL_TOTAL_RECIBO_
FROM
  `{}`
WHERE
  NR_RECIBO_SYSTUR IS NOT NULL
GROUP BY
  NR_RECIBO_SYSTUR)""".format(dataset)

In [ ]:
Valor_total_recibos = client.query(Valor_total_recibos).to_dataframe()
Valor_total_recibos

## Calculando Valor total Aéreo

In [ ]:
Valor_Total_Aereo="""
SELECT
  SUM(VL_REPASSE_FORNECEDOR) + SUM(VL_TAXA_EMBARQUE) AS VALOR_TOTAL_AEREO
FROM(
  SELECT
    VL_REPASSE_FORNECEDOR,
    VL_TAXA_EMBARQUE
  FROM
    `{}`
  WHERE
    TIPO_FORNECEDOR = 'AEREO'
    AND NR_RECIBO_SYSTUR IS NOT NULL )""".format(dataset)

In [ ]:
Valor_Total_Aereo = client.query(Valor_Total_Aereo).to_dataframe()
Valor_Total_Aereo

### Calculando o Valor Total Dos Recibos sem Aéreo

In [ ]:
Valor_total_recibos_sem_aereo="""
SELECT
  SUM(VL_TOTAL_RECIBO_) AS VALOR_TOTAL_RECIBOS_SEM_AEREO
FROM(
SELECT
  NR_RECIBO_SYSTUR,
  MAX(VL_TOTAL_RECIBO) AS VL_TOTAL_RECIBO_
FROM(
SELECT
  NR_RECIBO_SYSTUR,
  VL_TOTAL_RECIBO
FROM
 `{}`
WHERE
  bilhetes_t IS NULL
  AND NR_RECIBO_SYSTUR IS NOT NULL
  AND Tipo_fornecedor_t IS NULL)
GROUP BY
  NR_RECIBO_SYSTUR)""".format(dataset)

In [ ]:
Valor_total_recibos_sem_aereo = client.query(Valor_total_recibos_sem_aereo).to_dataframe()
Valor_total_recibos_sem_aereo

### Calculando o valor total dos recibos -> recibo complementar

In [ ]:
Valor_Total_Recibo_Complementar = """
SELECT
  SUM(VL_TOTAL_RECIBO_) AS VALOR_TOTAL_RECIBO_COMPLEMENTAR
FROM(
SELECT
  NR_RECIBO_SYSTUR,
  MAX(VL_TOTAL_RECIBO) AS VL_TOTAL_RECIBO_
FROM (
  SELECT
    NR_RECIBO_SYSTUR,
    VL_TOTAL_RECIBO
  FROM
    `{}`
  WHERE
    bilhetes_t IS NULL
    AND NR_RECIBO_SYSTUR IS NOT NULL
    AND Tipo_fornecedor_t = "AEREO")
GROUP BY
  NR_RECIBO_SYSTUR)""".format(dataset)

In [ ]:
Valor_Total_Recibo_Complementar = client.query(Valor_Total_Recibo_Complementar).to_dataframe()
Valor_Total_Recibo_Complementar

### Calculando valor de aéreo -> recibos complementares

In [ ]:
Aereo_Total_Recibo_Complementar = """
SELECT
  VL_REPASSE_FORNECEDOR_ + VL_TAXA_EMBARQUE_ AS AEREO_TOTAL_RECIBO_COMPLEMENTAR
FROM(
SELECT
  SUM(VL_REPASSE_FORNECEDOR) AS VL_REPASSE_FORNECEDOR_,
  SUM(VL_TAXA_EMBARQUE) AS VL_TAXA_EMBARQUE_
FROM (
  SELECT
    TIPO_FORNECEDOR,
    VL_REPASSE_FORNECEDOR,
    VL_TAXA_EMBARQUE
  FROM
    `{}`
  WHERE
    bilhetes_t IS NULL
    AND NR_RECIBO_SYSTUR IS NOT NULL
    AND Tipo_fornecedor_t = 'AEREO')
 WHERE
  TIPO_FORNECEDOR = "AEREO")""".format(dataset)

In [ ]:
Aereo_Total_Recibo_Complementar = client.query(Aereo_Total_Recibo_Complementar).to_dataframe()
Aereo_Total_Recibo_Complementar

In [ ]:
Valor_Total_Recibo_Voo_Regular = """
SELECT
  SUM(VL_TOTAL_RECIBO_) AS VALOR_TOTAL_RECIBO_VOO_REGULAR
FROM(
SELECT
  NR_RECIBO_SYSTUR,
  MAX(VL_TOTAL_RECIBO) AS VL_TOTAL_RECIBO_
FROM(
SELECT
  NR_RECIBO_SYSTUR,
  VL_TOTAL_RECIBO
FROM
  `{}`
WHERE
  bilhetes_t IS NOT NULL
  AND MALHA IS NULL
  AND NR_RECIBO_SYSTUR IS NOT NULL)
GROUP BY
  NR_RECIBO_SYSTUR)""".format(dataset)

In [ ]:
Valor_Total_Recibo_Voo_Regular = client.query(Valor_Total_Recibo_Voo_Regular).to_dataframe()
Valor_Total_Recibo_Voo_Regular

### Calculando valor de aéreo -> voos regulares

In [ ]:
Aereo_Total_Recibo_Voo_Regular = """
SELECT
  VL_REPASSE_FORNECEDOR_ + VL_TAXA_EMBARQUE_ AS AEREO_TOTAL_RECIBO_VOO_REGULAR
FROM(
SELECT
  SUM(VL_REPASSE_FORNECEDOR) AS VL_REPASSE_FORNECEDOR_,
  SUM(VL_TAXA_EMBARQUE) AS VL_TAXA_EMBARQUE_
FROM(
SELECT
  VL_REPASSE_FORNECEDOR,
  VL_TAXA_EMBARQUE
FROM
  `{}`
WHERE
  bilhetes_t IS NOT NULL
  AND MALHA IS NULL
  AND TIPO_FORNECEDOR = 'AEREO'
  AND NR_RECIBO_SYSTUR IS NOT NULL))""".format(dataset)

In [ ]:
Aereo_Total_Recibo_Voo_Regular = client.query(Aereo_Total_Recibo_Voo_Regular).to_dataframe()
Aereo_Total_Recibo_Voo_Regular

### Calculando valor total de recibos com voos fretados

In [ ]:
Valor_Total_Recibo_Voo_Fretado = """
SELECT
  SUM(VL_TOTAL_RECIBO_) AS VALOR_TOTAL_RECIBO_VOO_FRETADO
FROM(
SELECT
  NR_RECIBO_SYSTUR,
  MAX(VL_TOTAL_RECIBO) AS VL_TOTAL_RECIBO_
FROM (
  SELECT
    NR_RECIBO_SYSTUR,
    VL_TOTAL_RECIBO
  FROM
    `{}`
  WHERE
    bilhetes_t IS NOT NULL
    AND MALHA IS NOT NULL
    AND Localizadores_t IS NULL
    AND NR_RECIBO_SYSTUR IS NOT NULL)
GROUP BY
  NR_RECIBO_SYSTUR)""".format(dataset)

In [ ]:
Valor_Total_Recibo_Voo_Fretado = client.query(Valor_Total_Recibo_Voo_Fretado).to_dataframe()
Valor_Total_Recibo_Voo_Fretado

### Calculando Valor aéreo -> recibos voos fretados

In [ ]:
Aereo_Total_Recibo_Voo_Fretado = """
SELECT
  VL_REPASSE_FORNECEDOR_ + VL_TAXA_EMBARQUE_ AS AEREO_TOTAL_RECIBO_VOO_FRETADO
FROM(
SELECT
  SUM(VL_REPASSE_FORNECEDOR) AS VL_REPASSE_FORNECEDOR_,
  SUM(VL_TAXA_EMBARQUE) AS VL_TAXA_EMBARQUE_
FROM (
  SELECT
    TIPO_FORNECEDOR,
    VL_REPASSE_FORNECEDOR,
    VL_TAXA_EMBARQUE
  FROM
    `{}`
  WHERE
    bilhetes_t IS NOT NULL
    AND MALHA IS NOT NULL
    AND Localizadores_t IS NULL
    AND NR_RECIBO_SYSTUR IS NOT NULL)
WHERE
  TIPO_FORNECEDOR = 'AEREO')""".format(dataset)

In [ ]:
Aereo_Total_Recibo_Voo_Fretado = client.query(Aereo_Total_Recibo_Voo_Fretado).to_dataframe()
Aereo_Total_Recibo_Voo_Fretado

### Calculando valor total de recibos xxx Block Charter

In [ ]:
Valor_Total_Recibo_Voo_Block_Charter = """
SELECT
  SUM(VL_TOTAL_RECIBO_) AS VALOR_TOTAL_RECIBO_VOO_BLOCK_CHARTER
FROM(
SELECT
  NR_RECIBO_SYSTUR,
  MAX(VL_TOTAL_RECIBO) AS VL_TOTAL_RECIBO_
FROM(
  SELECT
    NR_RECIBO_SYSTUR,
    VL_TOTAL_RECIBO
  FROM
    `{}`
  WHERE
    bilhetes_t IS NOT NULL
    AND MALHA IS NOT NULL
    AND Localizadores_t IS NOT NULL
    AND NR_RECIBO_SYSTUR IS NOT NULL)
GROUP BY
  NR_RECIBO_SYSTUR)""".format(dataset)

In [ ]:
Valor_Total_Recibo_Voo_Block_Charter = client.query(Valor_Total_Recibo_Voo_Block_Charter).to_dataframe()
Valor_Total_Recibo_Voo_Block_Charter

### Calculando Valor aéreo -> recibos block charter

In [ ]:
Aereo_Total_Recibo_Voo_Block_Charter = """
SELECT
  VL_REPASSE_FORNECEDOR_ + VL_TAXA_EMBARQUE_ AS AEREO_TOTAL_RECIBO_VOO_BLOCK_CHARTER
FROM(
SELECT
  SUM(VL_REPASSE_FORNECEDOR) AS VL_REPASSE_FORNECEDOR_,
  SUM(VL_TAXA_EMBARQUE) AS VL_TAXA_EMBARQUE_
FROM(
  SELECT
    TIPO_FORNECEDOR,
    VL_REPASSE_FORNECEDOR,
    VL_TAXA_EMBARQUE
  FROM
    `{}`
  WHERE
    bilhetes_t IS NOT NULL
    AND MALHA IS NOT NULL
    AND Localizadores_t IS NOT NULL
    AND NR_RECIBO_SYSTUR IS NOT NULL)
WHERE
  TIPO_FORNECEDOR = 'AEREO')""".format(dataset)

In [ ]:
Aereo_Total_Recibo_Voo_Block_Charter = client.query(Aereo_Total_Recibo_Voo_Block_Charter).to_dataframe()
Aereo_Total_Recibo_Voo_Block_Charter

## Resultados Finais

In [ ]:
d = {'TIPO': ['PACOTES SEM VOO','VOOS REGULARES','BLOCK CHARTER','FRETAMENTO','RECIBOS COMPLEMENTARES','TOTAL'],
     'TOTAL RECIBOS': [Valor_total_recibos_sem_aereo.iloc[0][0],Valor_Total_Recibo_Voo_Regular.iloc[0][0],
                       Valor_Total_Recibo_Voo_Block_Charter.iloc[0][0],
                       Valor_Total_Recibo_Voo_Fretado.iloc[0][0],Valor_Total_Recibo_Complementar.iloc[0][0],0], 
     'TOTAL AEREO': [0, Aereo_Total_Recibo_Voo_Regular.iloc[0][0],Aereo_Total_Recibo_Voo_Block_Charter.iloc[0][0],
                    Aereo_Total_Recibo_Voo_Fretado.iloc[0][0],Aereo_Total_Recibo_Complementar.iloc[0][0],0]}
Resultado = pd.DataFrame(data=d)
Resultado = Resultado.set_index('TIPO')
cols = Resultado.columns.tolist()
cols = cols[-1:] + cols[:-1]
Resultado = Resultado[cols]
Resultado.loc['TOTAL']['TOTAL RECIBOS'] = sum(Resultado['TOTAL RECIBOS'])
Resultado.loc['TOTAL']['TOTAL AEREO'] = sum(Resultado['TOTAL AEREO'])
Resultado